# Daten einlesen

## Notwendige Bibliotheken importieren

In [165]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import numpy as np
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

## Daten laden und in DataFrame verbinden

In [166]:
# Download der Daten von URLs
umsatzdaten_url = "https://github.com/opencampus-sh/einfuehrung-in-data-science-und-ml/raw/main/umsatzdaten_gekuerzt.csv"
kiwodaten_url = "https://github.com/opencampus-sh/einfuehrung-in-data-science-und-ml/raw/main/kiwo.csv"
wetterdaten_url = "https://github.com/opencampus-sh/einfuehrung-in-data-science-und-ml/raw/main/wetter.csv"

# Laden der Daten in Pandas DataFrames
umsatzdaten = pd.read_csv(umsatzdaten_url)
kiwo = pd.read_csv(kiwodaten_url)
wetter = pd.read_csv(wetterdaten_url)

# Zusammenführen der Daten mit Outer Join
merged_data = pd.merge(umsatzdaten, kiwo, on='Datum', how='outer')
merged_data = pd.merge(merged_data, wetter, on='Datum', how='outer')

# Gib den zusammengeführten DataFrame aus
print(merged_data)

            Datum  Warengruppe  Umsatz  KielerWoche  Bewoelkung  Temperatur  \
0      2012-01-01          NaN     NaN          NaN         8.0      9.8250   
1      2012-01-02          NaN     NaN          NaN         7.0      7.4375   
2      2012-01-03          NaN     NaN          NaN         8.0      5.5375   
3      2012-01-04          NaN     NaN          NaN         4.0      5.6875   
4      2012-01-05          NaN     NaN          NaN         6.0      5.3000   
...           ...          ...     ...          ...         ...         ...   
10114  2019-07-28          NaN     NaN          NaN         3.0     23.3500   
10115  2019-07-29          NaN     NaN          NaN         6.0     25.2500   
10116  2019-07-30          NaN     NaN          NaN         7.0     20.7375   
10117  2019-07-31          NaN     NaN          NaN         6.0     20.4500   
10118  2019-08-01          NaN     NaN          NaN         5.0     21.0625   

       Windgeschwindigkeit  Wettercode  
0         

## Datum konvertieren / Wochentag / Feiertage hinzufügen

In [167]:
# Konvertieren der Spalte "Datum" in Datumsformat
merged_data["Datum"] = pd.to_datetime(merged_data["Datum"])

# Hinzufügen von numerischen Wochentag (Montag=0, Sonntag=6)
merged_data['Wochentag_Numerisch'] = merged_data['Datum'].dt.weekday

# Konvertieren von KielerWoche in binären Wert (1 falls nicht NaN, sonst 0)
merged_data['KielerWoche'] = merged_data['KielerWoche'].notna().astype(int)

# Hinzufügen von Wochenend-Spalte (Samstag und Sonntag)
merged_data['Wochenende'] = (merged_data['Datum'].dt.dayofweek >= 5).astype(int)

# Liste der Feiertage
feiertage = pd.to_datetime([
    '2012-01-01', '2012-04-06', '2012-04-09', '2012-05-01', '2012-05-17', '2012-05-28', '2012-10-03', '2012-12-25', '2012-12-26', '2013-01-01', '2013-03-29', '2013-04-01', '2013-05-01', '2013-05-09', '2013-05-20', '2013-10-03', '2013-12-25', '2013-12-26', '2014-01-01', '2014-04-18', '2014-04-21', '2014-05-01', '2014-05-29', '2014-06-09', '2014-10-03', '2014-12-25', '2014-12-26', '2015-01-01', '2015-04-03', '2015-04-06', '2015-05-01', '2015-05-14', '2015-05-25', '2015-10-03', '2015-12-25', '2015-12-26', '2016-01-01', '2016-03-25', '2016-03-28', '2016-05-01', '2016-05-05', '2016-05-16', '2016-10-03', '2016-12-25', '2016-12-26', '2017-01-01', '2017-04-14', '2017-04-17', '2017-05-01', '2017-05-25', '2017-06-05', '2017-10-03', '2017-12-25', '2017-12-26',
'2018-01-01', '2018-03-30', '2018-04-02', '2018-05-01', '2018-05-10', '2018-05-21', '2018-10-03', '2018-12-25', '2018-12-26', '2019-01-01', '2019-04-19', '2019-04-22', '2019-05-01', '2019-05-30', '2019-06-10', '2019-10-03', '2019-12-25', '2019-12-26'
] )

# Hinzufügen von Feiertags-Spalte (1 falls Feiertag, sonst 0)
merged_data['Feiertag'] = merged_data['Datum'].isin(feiertage).astype(int)

# Lagged variables für Feiertage und Wochenende (1 Tag vorher)
merged_data['TagVorherFeiertag'] = merged_data['Feiertag'].shift(1).fillna(0).astype(int)
merged_data['TagVorherWochenende'] = merged_data['Wochenende'].shift(1).fillna(0).astype(int)

# Lagged variable für den Umsatz im Vergleich zum Vortag
merged_data['UmsatzVortag'] = merged_data['Umsatz'].shift(1).fillna(0)

print(merged_data)

           Datum  Warengruppe  Umsatz  KielerWoche  Bewoelkung  Temperatur  \
0     2012-01-01          NaN     NaN            0         8.0      9.8250   
1     2012-01-02          NaN     NaN            0         7.0      7.4375   
2     2012-01-03          NaN     NaN            0         8.0      5.5375   
3     2012-01-04          NaN     NaN            0         4.0      5.6875   
4     2012-01-05          NaN     NaN            0         6.0      5.3000   
...          ...          ...     ...          ...         ...         ...   
10114 2019-07-28          NaN     NaN            0         3.0     23.3500   
10115 2019-07-29          NaN     NaN            0         6.0     25.2500   
10116 2019-07-30          NaN     NaN            0         7.0     20.7375   
10117 2019-07-31          NaN     NaN            0         6.0     20.4500   
10118 2019-08-01          NaN     NaN            0         5.0     21.0625   

       Windgeschwindigkeit  Wettercode  Wochentag_Numerisch  Wo

# Deskriptive Statistiken berechnen

In [168]:
# Temperatur
temp_stats = merged_data['Temperatur'].describe()

# Umsatz
umsatz_stats = merged_data['Umsatz'].describe()

# Wind
wind_stats = merged_data['Windgeschwindigkeit'].describe()

# Umsatz je Warengruppe
deskriptive_statistiken = merged_data.groupby("Warengruppe")["Umsatz"].describe()

## Output deskriptive Statistiken

In [169]:
# Output der Deskriptiven Statistiken
print("Deskriptive Statistiken für Temperatur:")
print(temp_stats)

print("\nDeskriptive Statistiken für Umsatz:")
print(umsatz_stats)

print("\nDeskriptive Statistiken für Windgeschwindigkeit:")
print(wind_stats)

# Ausgabe der deskriptiven Statistiken
print(deskriptive_statistiken)

Deskriptive Statistiken für Temperatur:
count    10103.000000
mean        12.014560
std          7.212466
min        -10.250000
25%          6.250000
50%         11.625000
75%         17.875000
max         32.671428
Name: Temperatur, dtype: float64

Deskriptive Statistiken für Umsatz:
count    9334.000000
mean      206.749044
std       144.545189
min         7.051201
25%        96.897441
50%       161.900831
75%       280.644663
max      1879.461831
Name: Umsatz, dtype: float64

Deskriptive Statistiken für Windgeschwindigkeit:
count    10103.000000
mean        11.026527
std          4.131774
min          3.000000
25%          8.000000
50%         10.000000
75%         13.000000
max         35.000000
Name: Windgeschwindigkeit, dtype: float64
              count        mean         std         min         25%  \
Warengruppe                                                           
1.0          1819.0  122.142509   40.299209   23.108406   96.436728   
2.0          1819.0  402.497911  139

# Korrelationen berechnen

In [170]:
# Korrelationen berechnen
corr_umsatz_weekend = merged_data['Umsatz'].corr(merged_data['Wochenende'].astype(int))
corr_umsatz_weekday = merged_data.groupby('Wochentag_Numerisch')['Umsatz'].mean()
corr_umsatz_temp = merged_data['Umsatz'].corr(merged_data['Temperatur'])
corr_umsatz_wind = merged_data['Umsatz'].corr(merged_data['Windgeschwindigkeit'])

# Korrelationsmatrix für Umsatzdaten
corr_matrix = merged_data.corr()

# Korrelationen zwischen Umsatz und einzelnen Wochentagen berechnen
corr_umsatz_weekdays = {}
for day in range(7):
    # Filtere den DataFrame nach dem aktuellen Wochentag
    day_data = merged_data[merged_data['Wochentag_Numerisch'] == day]
    # Berechne die Korrelation zwischen Umsatz und dem aktuellen Wochentag
    corr = day_data['Umsatz'].corr(day_data['Wochentag_Numerisch'])
    # Speichere die Korrelation für den aktuellen Wochentag
    corr_umsatz_weekdays[day] = corr

# Ausgabe der Korrelationen für die einzelnen Wochentage
for day, corr in corr_umsatz_weekdays.items():
    print(f"Korrelation zwischen Umsatz und {day}: {corr}")

# Berechne die Kontingenztabelle
kontingenztabelle = pd.crosstab(merged_data["Warengruppe"], merged_data["Wochenende"])

# Berechne Cramér's V
chi2 = stats.chi2_contingency(kontingenztabelle)[0]
n = kontingenztabelle.sum().sum()
min_dim = min(kontingenztabelle.shape) - 1
cramers_v = np.sqrt(chi2 / (n * min_dim))

print(f"Cramér's V: {cramers_v}")

# Optional: Ausgabe der Kontingenztabelle zur Überprüfung
print(kontingenztabelle)

Korrelation zwischen Umsatz und 0: nan
Korrelation zwischen Umsatz und 1: nan
Korrelation zwischen Umsatz und 2: nan
Korrelation zwischen Umsatz und 3: nan
Korrelation zwischen Umsatz und 4: nan
Korrelation zwischen Umsatz und 5: nan
Korrelation zwischen Umsatz und 6: nan
Cramér's V: 0.002742493155364742
Wochenende      0    1
Warengruppe           
1.0          1298  521
2.0          1298  521
3.0          1298  521
4.0          1255  511
5.0          1298  521
6.0           209   83


/home/codespace/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/codespace/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


# Lineares Modelle

## lineares Modell 1

In [171]:
# Fit the linear model including categorical variables and additional features
mod = smf.ols('Umsatz ~ C(Warengruppe) + Temperatur + Wochenende + KielerWoche + C(Wochentag_Numerisch)  ', data=merged_data).fit()

# Print the summary
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:                 Umsatz   R-squared:                       0.713
Model:                            OLS   Adj. R-squared:                  0.712
Method:                 Least Squares   F-statistic:                     1775.
Date:                Wed, 19 Jun 2024   Prob (F-statistic):               0.00
Time:                        18:42:20   Log-Likelihood:                -53757.
No. Observations:                9318   AIC:                         1.075e+05
Df Residuals:                    9304   BIC:                         1.076e+05
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept         

## lineares modell 2

In [172]:
mod = smf.ols('Umsatz ~ C(Warengruppe) + Temperatur + Wochenende + KielerWoche  + C(Wochentag_Numerisch)', data=merged_data).fit()

# Print the summary
print(mod.summary())



                            OLS Regression Results                            
Dep. Variable:                 Umsatz   R-squared:                       0.713
Model:                            OLS   Adj. R-squared:                  0.712
Method:                 Least Squares   F-statistic:                     1775.
Date:                Wed, 19 Jun 2024   Prob (F-statistic):               0.00
Time:                        18:42:20   Log-Likelihood:                -53757.
No. Observations:                9318   AIC:                         1.075e+05
Df Residuals:                    9304   BIC:                         1.076e+05
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept         

## lineares modell 3

In [173]:
# Baue das Modell, inklusive Interaktionsterm zwischen Warengruppe und Wochentag
mod = smf.ols('Umsatz ~ C(Warengruppe) * C(Wochentag_Numerisch) + Temperatur + Wochenende + KielerWoche', data=merged_data).fit()

# Print the summary
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:                 Umsatz   R-squared:                       0.747
Model:                            OLS   Adj. R-squared:                  0.746
Method:                 Least Squares   F-statistic:                     637.3
Date:                Wed, 19 Jun 2024   Prob (F-statistic):               0.00
Time:                        18:42:20   Log-Likelihood:                -53162.
No. Observations:                9318   AIC:                         1.064e+05
Df Residuals:                    9274   BIC:                         1.067e+05
Df Model:                          43                                         
Covariance Type:            nonrobust                                         
                                                        coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------

## lineares modell 4

In [174]:
# Baue das Modell, inklusive Interaktionstermen zwischen Warengruppe und Wochentag sowie Warengruppe und KielerWoche
mod = smf.ols('Umsatz ~ C(Warengruppe) * C(Wochentag_Numerisch) + Temperatur + Wochenende + C(Warengruppe) * KielerWoche', data=merged_data).fit()

# Print the summary
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:                 Umsatz   R-squared:                       0.749
Model:                            OLS   Adj. R-squared:                  0.748
Method:                 Least Squares   F-statistic:                     589.2
Date:                Wed, 19 Jun 2024   Prob (F-statistic):               0.00
Time:                        18:42:21   Log-Likelihood:                -53124.
No. Observations:                9318   AIC:                         1.063e+05
Df Residuals:                    9270   BIC:                         1.067e+05
Df Model:                          47                                         
Covariance Type:            nonrobust                                         
                                                        coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------

## lineares modell 5

In [175]:
# Fit the linear model including categorical variables and additional features
mod = smf.ols('Umsatz ~ C(Warengruppe) + Temperatur + Wochenende + KielerWoche + C(Wochentag_Numerisch)  ', data=merged_data).fit()

# Print the summary
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:                 Umsatz   R-squared:                       0.713
Model:                            OLS   Adj. R-squared:                  0.712
Method:                 Least Squares   F-statistic:                     1775.
Date:                Wed, 19 Jun 2024   Prob (F-statistic):               0.00
Time:                        18:42:21   Log-Likelihood:                -53757.
No. Observations:                9318   AIC:                         1.075e+05
Df Residuals:                    9304   BIC:                         1.076e+05
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept         

## lineares modell 6 -> bestes Ergebnis bisher

In [176]:
# Statsmodels Modell mit Interaktionen und Lagged variables
mod = smf.ols('Umsatz ~ C(Warengruppe) * C(Wochentag_Numerisch) + Temperatur + Wochenende + C(Warengruppe) * KielerWoche + TagVorherFeiertag * C(Warengruppe) + TagVorherWochenende * C(Warengruppe) + UmsatzVortag * C(Warengruppe)', data=merged_data).fit()

# Modellzusammenfassung anzeigen
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:                 Umsatz   R-squared:                       0.814
Model:                            OLS   Adj. R-squared:                  0.813
Method:                 Least Squares   F-statistic:                     697.5
Date:                Wed, 19 Jun 2024   Prob (F-statistic):               0.00
Time:                        18:42:21   Log-Likelihood:                -51738.
No. Observations:                9318   AIC:                         1.036e+05
Df Residuals:                    9259   BIC:                         1.040e+05
Df Model:                          58                                         
Covariance Type:            nonrobust                                         
                                                        coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------

# Aufteilung der Daten

In [177]:
# Aufteilen in Trainings- und Validierungsdaten (70:30)
train_data, validation_data = train_test_split(merged_data, test_size=0.3, random_state=42)


# Training des Modell

## Fehlende Werte 

In [178]:
# Anzeige der Anzahl der fehlenden Werte in jeder Spalte
missing_values = merged_data.isnull().sum()
print(missing_values)

# Anzeige der Gesamtanzahl der fehlenden Werte
total_missing_values = merged_data.isnull().sum().sum()
print(f"Gesamtanzahl der fehlenden Werte: {total_missing_values}")

Datum                     0
Warengruppe             785
Umsatz                  785
KielerWoche               0
Bewoelkung               71
Temperatur               16
Windgeschwindigkeit      16
Wettercode             2538
Wochentag_Numerisch       0
Wochenende                0
Feiertag                  0
TagVorherFeiertag         0
TagVorherWochenende       0
UmsatzVortag              0
dtype: int64
Gesamtanzahl der fehlenden Werte: 4211


In [179]:
# Imputation der fehlenden Werte mit Mittelwert im Trainingsdatensatz
train_data_imputed = train_data.copy()
numerische_spalten = train_data.select_dtypes(include=np.number).columns
train_data_imputed[numerische_spalten] = train_data[numerische_spalten].fillna(train_data[numerische_spalten].mean())

In [180]:
# Neues Statsmodels Modell nach Imputation
mod = smf.ols(formula, data=train_data_imputed).fit()

# Modellzusammenfassung anzeigen
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:                 Umsatz   R-squared:                       0.830
Model:                            OLS   Adj. R-squared:                  0.828
Method:                 Least Squares   F-statistic:                     496.6
Date:                Wed, 19 Jun 2024   Prob (F-statistic):               0.00
Time:                        18:42:21   Log-Likelihood:                -38680.
No. Observations:                7083   AIC:                         7.750e+04
Df Residuals:                    7013   BIC:                         7.798e+04
Df Model:                          69                                         
Covariance Type:            nonrobust                                         
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------

# Vorhersage

In [181]:
# Einlesen der CSV-Dateien aus dem Ordner "Testdaten Kaggle"
wetter = pd.read_csv('Testdaten Kaggle/wetter.csv')
kiwo = pd.read_csv('Testdaten Kaggle/kiwo.csv')
test = pd.read_csv('Testdaten Kaggle/test.csv')
train = pd.read_csv('Testdaten Kaggle/train.csv')

# Merge der DataFrames train, wetter und kiwo
test_data = pd.merge(kiwo, wetter, on='Datum', how='outer')
test_data = pd.merge(test_data, train, on='Datum', how='outer')

print(test_data)

            Datum  KielerWoche  Bewoelkung  Temperatur  Windgeschwindigkeit  \
0      2012-01-01          NaN         8.0      9.8250                 14.0   
1      2012-01-02          NaN         7.0      7.4375                 12.0   
2      2012-01-03          NaN         8.0      5.5375                 18.0   
3      2012-01-04          NaN         4.0      5.6875                 19.0   
4      2012-01-05          NaN         6.0      5.3000                 23.0   
...           ...          ...         ...         ...                  ...   
10114  2019-07-28          NaN         3.0     23.3500                 14.0   
10115  2019-07-29          NaN         6.0     25.2500                  7.0   
10116  2019-07-30          NaN         7.0     20.7375                  8.0   
10117  2019-07-31          NaN         6.0     20.4500                  7.0   
10118  2019-08-01          NaN         5.0     21.0625                  9.0   

       Wettercode  id  Warengruppe  Umsatz  
0     

## Vorbereitung

In [182]:
# Konvertieren der Spalte "Datum" in Datumsformat für test_data
test_data['Datum'] = pd.to_datetime(test_data['Datum'])

# Hinzufügen von numerischen Wochentag (Montag=0, Sonntag=6)
test_data['Wochentag_Numerisch'] = test_data['Datum'].dt.weekday

# Konvertieren von KielerWoche in binären Wert (1 falls nicht NaN, sonst 0)
test_data['KielerWoche'] = test_data['KielerWoche'].notna().astype(int)

# Hinzufügen von Wochenend-Spalte (Samstag und Sonntag)
test_data['Wochenende'] = (test_data['Datum'].dt.dayofweek >= 5).astype(int)

# Liste der Feiertage
feiertage = pd.to_datetime([
    '2012-01-01', '2012-04-06', '2012-04-09', '2012-05-01', '2012-05-17', '2012-05-28', '2012-10-03', '2012-12-25', '2012-12-26', 
    '2013-01-01', '2013-03-29', '2013-04-01', '2013-05-01', '2013-05-09', '2013-05-20', '2013-10-03', '2013-12-25', '2013-12-26', 
    '2014-01-01', '2014-04-18', '2014-04-21', '2014-05-01', '2014-05-29', '2014-06-09', '2014-10-03', '2014-12-25', '2014-12-26', 
    '2015-01-01', '2015-04-03', '2015-04-06', '2015-05-01', '2015-05-14', '2015-05-25', '2015-10-03', '2015-12-25', '2015-12-26', 
    '2016-01-01', '2016-03-25', '2016-03-28', '2016-05-01', '2016-05-05', '2016-05-16', '2016-10-03', '2016-12-25', '2016-12-26', 
    '2017-01-01', '2017-04-14', '2017-04-17', '2017-05-01', '2017-05-25', '2017-06-05', '2017-10-03', '2017-12-25', '2017-12-26',
    '2018-01-01', '2018-03-30', '2018-04-02', '2018-05-01', '2018-05-10', '2018-05-21', '2018-10-03', '2018-12-25', '2018-12-26', 
    '2019-01-01', '2019-04-19', '2019-04-22', '2019-05-01', '2019-05-30', '2019-06-10', '2019-10-03', '2019-12-25', '2019-12-26'
])

# Hinzufügen von Feiertags-Spalte (1 falls Feiertag, sonst 0)
test_data['Feiertag'] = test_data['Datum'].isin(feiertage).astype(int)

# Lagged variables für Feiertage und Wochenende (1 Tag vorher)
test_data['TagVorherFeiertag'] = test_data['Feiertag'].shift(1).fillna(0).astype(int)
test_data['TagVorherWochenende'] = test_data['Wochenende'].shift(1).fillna(0).astype(int)

# Lagged variable für den Umsatz im Vergleich zum Vortag
test_data['UmsatzVortag'] = test_data['Umsatz'].shift(1).fillna(0)

# Statsmodels Modellformel und Modell mit Interaktionen und Lagged variables
formula = 'Umsatz ~ C(Warengruppe) * C(Wochentag_Numerisch) + Temperatur + Wochenende + C(Warengruppe) * KielerWoche + TagVorherFeiertag * C(Warengruppe) + TagVorherWochenende * C(Warengruppe) + UmsatzVortag * C(Warengruppe)'
mod = smf.ols(formula, data=merged_data).fit()

# Funktion zur Vorhersage für jedes Datum im test DataFrame
def predict_for_test_data(test_data):
    predictions = {}
    for index, row in test_data.iterrows():
        neue_daten = pd.DataFrame({
            'Warengruppe': [row['Warengruppe']],
            'Wochentag_Numerisch': row['Wochentag_Numerisch'],
            'Temperatur': row['Temperatur'],
            'Wochenende': row['Wochenende'],
            'KielerWoche': row['KielerWoche'],
            'TagVorherFeiertag': row['TagVorherFeiertag'],
            'TagVorherWochenende': row['TagVorherWochenende'],
            'UmsatzVortag': row['UmsatzVortag']
        })
        
        # Vorhersage machen
        prediction = mod.predict(neue_daten)
        predictions[index] = prediction.values[0]
    
    return predictions

# Vorhersage für alle Zeilen im test DataFrame machen
predictions = predict_for_test_data(test)

# Ausgabe der Vorhersagen
for index, umsatz in predictions.items():
    print(f'Vorhergesagter Umsatz für Zeile {index}: {umsatz}')


KeyError: 'Wochentag_Numerisch'